In [1]:
import re
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer
from tqdm import tqdm

c:\Users\fajar\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
# data = pd.read_csv("data/dataset_penyisihan_bdc_2024.csv", delimiter=";")
data = pd.read_csv("augmentedDataset1.csv", delimiter=",")

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23677 entries, 0 to 23676
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        23677 non-null  object
 1   label       23677 non-null  object
 2   clean_text  23677 non-null  object
dtypes: object(3)
memory usage: 555.1+ KB


In [32]:
data.head()

,text,label,clean_text
0,Kunjungan Prabowo ini untuk meresmikan dan menyerahkan proyek bantuan air bersih di lima titik. #IndonesiaSentris #IndonesiaHijau #02Melanjutkan #AnakMudaIndonesiaEmas Prabowo Subianto,Sumber Daya Alam,kunjungan prabowo ini untuk meresmikan dan menyerahkan proyek bantuan air bersih di lima titik. prabowo subianto
1,RT Anies dapat tepuk tangan meriah saat jadi Rektor mewajibkan mata kuliah anti-korupsi untuk memutus mata rantai korupsi.. #AminMiskinkanKoruptor https://t.co/hgXsoQGaEa [RE ekowboy2],Politik,anies dapat tepuk tangan meriah saat jadi rektor mewajibkan mata kuliah anti-korupsi untuk memutus mata rantai korupsi..
2,"@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= emng bener sih, pendukung 01 ada yg goblok, begitu jg dg pendukung 02.. hnya sj menurut pak Ridwan Kamil skemanya terbalik, klo 01 MAYORITAS PENDIDIKAN MENENGAH ATAS (artinya ada jg pendidikan rendah yg milih",Demografi,"emng bener sih, pendukung 01 ada yg goblok, begitu jg dg pendukung 02.. hnya sj menurut pak ridwan kamil skemanya terbalik, klo 01 mayoritas pendidikan menengah atas (artinya ada jg pendidikan rendah yg milih"
3,"RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7Q= Sewaktu anies bersikap kritis ke kinerja pak prabowo dianggap engga sopan karena dianggap kurang menghormati orang tua, giliran skrg gibran yg tengil dan sok kritis malah dianggap kritis dan keras. Apakah ini tidak standar ganda? Ã°ÂŸÂ¤Â­ [RE coldthem]",Politik,"sewaktu anies bersikap kritis ke kinerja pak prabowo dianggap engga sopan karena dianggap kurang menghormati orang tua, giliran skrg gibran yg tengil dan sok kritis malah dianggap kritis dan keras. apakah ini tidak standar ganda?"
4,Anies Baswedan Harap ASN termasuk TNI dan Polri Pegang Sumpahnya dalam Pemilu https://t.co/QRQZ3wI8YP,Politik,anies baswedan harap asn termasuk tni dan polri pegang sumpahnya dalam pemilu


In [33]:
print(data['label'].unique())
print(data.label.value_counts())

['Sumber Daya Alam' 'Politik' 'Demografi' 'Pertahanan dan Keamanan'
 'Ideologi' 'Ekonomi' 'Sosial Budaya' 'Geografi']
Politik                    2969
Geografi                   2969
Pertahanan dan Keamanan    2966
Ideologi                   2964
Ekonomi                    2956
Sosial Budaya              2956
Demografi                  2954
Sumber Daya Alam           2943
Name: label, dtype: int64


## Cleaning Data

In [34]:
import unicodedata
import string
# Fungsi membersihkan teks tanpa stemming/lemmatisasi
def clean_text(text):
    text = str(text).lower()

    # 1. Hapus RT, via, cc di awal
    text = re.sub(r'^(RT|rt|via|cc)\b', '', text).strip()

    # 2. Hapus mention @username
    text = re.sub(r'@\w+', '', text)

    # 3. Hapus URL
    text = re.sub(r'http\S+', '', text)

    # 4. Hapus hashtag
    text = re.sub(r'#\S+', '', text)

    # 5. Hapus bracket [RE ...] atau yang sejenis
    text = re.sub(r'\[.*?\]', '', text)

    # 6. Hapus encoding random (+ECNv...= dsb)
    text = re.sub(r'\S*=\S*', '', text)

    # 7. Hapus karakter non-ASCII dan simbol aneh
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)

    # 8. Normalisasi unicode (hilangkan diakritik tak perlu)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # 9. Hapus tanda baca
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)

    # 10. Ganti & menjadi "dan"
    text = text.replace("&", " dan ")

    # 11. Hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Terapkan ke kolom data
data['clean_text'] = data['clean_text'].apply(clean_text)


In [35]:
# Hapus baris jika kolom 'clean_text' kosong atau hanya berisi spasi
data = data[data['clean_text'].astype(str).str.strip() != ""]

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_text(text):
    return stemmer.stem(text)

# 2. Terapkan stemming
data['stemmed_text'] = data['clean_text'].apply(stem_text)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4993 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        4993 non-null   object
 1   label       4993 non-null   object
 2   clean_text  4993 non-null   object
dtypes: object(3)
memory usage: 156.0+ KB


In [37]:
data.to_csv("data/prepData3.csv", index=False)